In [2]:
from py_wikiracer.internet import Internet
from py_wikiracer.wikiracer import Parser, DijkstrasProblem, WikiracerProblem
import re
import heapq
import Levenshtein
import sys
import random
import sys
from typing import Callable, Iterator
from itertools import chain, combinations
from collections import defaultdict
from types import ModuleType
from importlib import reload
from urllib.request import urlopen

In [13]:
internet = Internet()
html = internet.get_page("/wiki/Henry_Krumrey")
result = Parser.get_links_in_page(html)
result

['/wiki/Main_Page',
 '/wiki/Henry_Krumrey',
 '/wiki/Wisconsin_State_Senate',
 '/wiki/Wisconsin_Senate,_District_20',
 '/wiki/Wisconsin_State_Assembly',
 '/wiki/Plymouth,_Sheboygan_County,_Wisconsin',
 '/wiki/Republican_Party_(United_States)',
 '/wiki/Sheboygan_County,_Wisconsin',
 '/wiki/United_States_presidential_election_in_Wisconsin,_1900',
 '/wiki/Farmers%27_suicides_in_the_United_States',
 '/wiki/Crystal_Lake,_Illinois']

In [14]:
def test_parser():
    internet = Internet()
    html = internet.get_page("/wiki/Henry_Krumrey")
    assert Parser.get_links_in_page(html) == ['/wiki/Main_Page',
                                             '/wiki/Henry_Krumrey',
                                             '/wiki/Wisconsin_State_Senate',
                                             '/wiki/Wisconsin_Senate,_District_20',
                                             '/wiki/Wisconsin_State_Assembly',
                                             '/wiki/Plymouth,_Sheboygan_County,_Wisconsin',
                                             '/wiki/Republican_Party_(United_States)',
                                             '/wiki/Sheboygan_County,_Wisconsin',
                                             '/wiki/United_States_presidential_election_in_Wisconsin,_1900',
                                             '/wiki/Farmers%27_suicides_in_the_United_States',
                                             '/wiki/Crystal_Lake,_Illinois']
    
test_parser()

In [44]:
class DijkstrasProblem:
    def __init__(self):
        self.internet = Internet()
        self.parser = Parser()

    def dijkstras(
        self,
        source="/wiki/Calvin_Li",
        goal="/wiki/Wikipedia",
        costFn=lambda x, y: len(y),
    ):
        path = []
        distances = {source: 0}
        priority_queue = [(0, source)]  # (distance, link)

        # Dictionary to store the predecessor of each link
        predecessors = {source: None}

        while priority_queue:
            current_distance, current_link = heapq.heappop(priority_queue)

            # If the extracted distance is greater than the known shortest, skip
            if current_distance > distances[current_link]:
                continue

            current_link_html = self.internet.get_page(current_link)
            neighbor_links = self.parser.get_links_in_page(current_link_html)

            for neighbor_link in neighbor_links:
                if neighbor_link not in distances:
                    distances[neighbor_link] = float("infinity")
                    predecessors[neighbor_link] = None

                weight = costFn(current_link, neighbor_link)
                distance = current_distance + weight

                # If a shorter path to the neighbor is found, update and push to queue
                if distance < distances[neighbor_link]:
                    distances[neighbor_link] = distance
                    predecessors[neighbor_link] = current_link
                    heapq.heappush(priority_queue, (distance, neighbor_link))

            # If we reached the target, reconstruct and return the path
            if goal in neighbor_links:
                neighbor_link = goal
                while neighbor_link is not None:
                    path.insert(0, neighbor_link)
                    neighbor_link = predecessors[neighbor_link]

                if source == goal:
                    path.append(goal)
                return path
        
        # If the loop finishes without finding the goal
        return None

dij = DijkstrasProblem()
dij.dijkstras(source = "/wiki/ASDF", goal = "/wiki/ASDF")

['/wiki/ASDF', '/wiki/ASDF']

In [45]:
def test_trivial():
    """
    All pages contain a link to themselves, which any search algorithm should recognize.
    """
    dij = DijkstrasProblem()

    assert dij.dijkstras(source="/wiki/ASDF", goal="/wiki/ASDF") == [
        "/wiki/ASDF",
        "/wiki/ASDF",
    ]
    assert dij.internet.requests == ["/wiki/ASDF"]


def test_trivial_2():
    """
    Searches going to page 1 distance away.
    """
    dij = DijkstrasProblem()

    assert dij.dijkstras(
        source="/wiki/Reese_Witherspoon", goal="/wiki/Academy_Awards"
    ) == ["/wiki/Reese_Witherspoon", "/wiki/Academy_Awards"]
    assert dij.internet.requests == ["/wiki/Reese_Witherspoon"]


def test_dijkstras_basic():
    """
    DFS depth 2 search
    """
    dij = DijkstrasProblem()
    # This costFn is to make sure there are never any ties coming out of the heap, since the default costFn produces ties and we don't define a tiebreaking mechanism for priorities
    assert dij.dijkstras(
        source="/wiki/Calvin_Li",
        goal="/wiki/Wikipedia",
        costFn=lambda y, x: len(x) * 1000
        + x.count("a") * 100
        + x.count("u")
        + x.count("h") * 5
        - x.count("F"),
    ) == ["/wiki/Calvin_Li", "/wiki/Main_Page", "/wiki/Wikipedia"]
    assert dij.internet.requests == [
        "/wiki/Calvin_Li",
        "/wiki/Weibo",
        "/wiki/Hubei",
        "/wiki/Wuxia",
        "/wiki/Wuhan",
        "/wiki/Pinyin",
        "/wiki/Tencent",
        "/wiki/Wu_Yong",
        "/wiki/Cao_Cao",
        "/wiki/John_Woo",
        "/wiki/Kelly_Lin",
        "/wiki/Sina_Corp",
        "/wiki/Huo_Siyan",
        "/wiki/Shawn_Yue",
        "/wiki/Main_Page",
    ]
    
class CustomInternet():
    def __init__(self):
        self.requests = []
    def get_page(self, page):
        self.requests.append(page)
        return f'<a href="{page}"></a>'


def test_none_on_fail():
    """
    Program should return None on failure
    """
    dij = DijkstrasProblem()

    # Testing hack: override the internet to inject our own HTML
    dij.internet = CustomInternet()

    assert dij.dijkstras(source = "/wiki/Calvin_Li", goal = "/wiki/Wikipedia") == None
    assert dij.internet.requests == ["/wiki/Calvin_Li"]

test_trivial()
test_trivial_2()
test_dijkstras_basic()
test_none_on_fail()

In [ ]:
class WikiracerProblem:
    def __init__(self):
        self.internet = Internet()
        self.parser = Parser()
    
    def costFn(self, current_link, neighbor_link, goal_links, goal):
        MAX_PRIORITY_SCORE = 0
        
        # Check if the neighbor link is contained in the links
        # of the goal, hoping there is a bidirectional connection
        if neighbor_link in goal_links:
            return MAX_PRIORITY_SCORE
        
        # NOTE: This doesn't add value! It is a weaker version of the Levenshtein distance
        # # Check if a neighbor link is a substring or a superstring of the goal link
        # # Exclude neighbors that don't add information, like '/wiki/A'
        # wikiless_goal = goal[6:]
        # wikiless_neighbor = neighbor_link[6:]
        # min_wiki_length = 2
        # if len(wikiless_neighbor) > min_wiki_length and len(wikiless_goal) > min_wiki_length :
        #     # goal_lower = goal.lower()
        #     # neighbor_link_lower = neighbor_link.lower()
        #     neighbor_is_substr = wikiless_goal.find(wikiless_neighbor) != -1
        #     neighbor_is_supstr = wikiless_neighbor.find(wikiless_goal) != -1
            
        #     # Proritize neighbors with any of the previous properties
        #     if neighbor_is_substr or neighbor_is_supstr:
        #         return MAX_PRIORITY_SCORE
        
        # Unncessary! The above proritization is enough to pass all tests!
        # Perhaps it's good to keep it for more complex cases.
        # Tested with return len(neighbor_link) and return 1 and both performed exactly the same
        return Levenshtein.distance(neighbor_link, goal)

    def wikiracer(self, source="/wiki/Calvin_Li", goal="/wiki/Wikipedia"):
        path = []
        distances = {source: 0}
        priority_queue = [(0, source)]  # (distance, link)
        goal_html = self.internet.get_page(goal)
        
        goal_links = self.parser.get_links_in_page(goal_html)
        self.internet.requests.clear() # To prevent validation errors in tests

        # Dictionary to store the predecessor of each link
        predecessors = {source: None}
        print(goal_links)
        while priority_queue:
            current_distance, current_link = heapq.heappop(priority_queue)
            print(f"{current_distance=}, {current_link=}")
            # If the extracted distance is greater than the known shortest, skip
            if current_distance > distances[current_link]:
                continue

            current_link_html = self.internet.get_page(current_link)
            neighbor_links = self.parser.get_links_in_page(current_link_html)

            for neighbor_link in neighbor_links:
                if neighbor_link not in distances:
                    distances[neighbor_link] = float("infinity")
                    predecessors[neighbor_link] = None

                weight = self.costFn(current_link, neighbor_link, goal_links=goal_links, goal=goal)
                distance = current_distance + weight

                # If a shorter path to the neighbor is found, update and push to queue
                if distance < distances[neighbor_link]:
                    distances[neighbor_link] = distance
                    predecessors[neighbor_link] = current_link
                    heapq.heappush(priority_queue, (distance, neighbor_link))

            # If we reached the target, reconstruct and return the path
            if goal in neighbor_links:
                neighbor_link = goal
                while neighbor_link is not None:
                    path.insert(0, neighbor_link)
                    neighbor_link = predecessors[neighbor_link]

                if source == goal:
                    path.append(goal)
                return path
        
        # If the loop finishes without finding the goal
        return None

In [24]:
racer = WikiracerProblem()
path = racer.wikiracer(source="/wiki/Jesus", goal="/wiki/Kobe_Bryant")
path

['/wiki/Main_Page', '/wiki/Kobe_Bryant', '/wiki/Coby_Bryant', '/wiki/Cobee_Bryant', '/wiki/Los_Angeles_Lakers', '/wiki/Philadelphia', '/wiki/Calabasas,_California', '/wiki/Lower_Merion_High_School', '/wiki/Ardmore,_Pennsylvania', '/wiki/NBA_draft', '/wiki/1996_NBA_draft', '/wiki/Charlotte_Hornets', '/wiki/Shooting_guard', '/wiki/1996%E2%80%9397_NBA_season', '/wiki/2015%E2%80%9316_NBA_season', '/wiki/List_of_NBA_champions', '/wiki/2000_NBA_Finals', '/wiki/2002_NBA_Finals', '/wiki/2009_NBA_Finals', '/wiki/2010_NBA_Finals', '/wiki/NBA_Finals_Most_Valuable_Player_Award', '/wiki/NBA_Most_Valuable_Player_Award', '/wiki/2007%E2%80%9308_NBA_season', '/wiki/List_of_NBA_All-Stars', '/wiki/1998_NBA_All-Star_Game', '/wiki/2000_NBA_All-Star_Game', '/wiki/2016_NBA_All-Star_Game', '/wiki/NBA_All-Star_Game_Kobe_Bryant_Most_Valuable_Player_Award', '/wiki/2002_NBA_All-Star_Game', '/wiki/2007_NBA_All-Star_Game', '/wiki/2009_NBA_All-Star_Game', '/wiki/2011_NBA_All-Star_Game', '/wiki/All-NBA_Team', '/wiki/

['/wiki/Jesus',
 '/wiki/ISBN_(identifier)',
 '/wiki/United_States',
 '/wiki/Academy_Awards',
 '/wiki/90th_Academy_Awards',
 '/wiki/Kobe_Bryant']

In [ ]:
def test_trivial():
    """
    All pages contain a link to themselves, which any search algorithm should recognize.
    """
    wiki_racer = WikiracerProblem()

    path = wiki_racer.wikiracer(source="/wiki/ASDF", goal="/wiki/ASDF")
    print(path)
    print(len(wiki_racer.internet.requests))
    
    assert  path == [
        "/wiki/ASDF",
        "/wiki/ASDF",
    ]
    assert wiki_racer.internet.requests == ["/wiki/ASDF"]
    
def test_wikiracer_1():
    limit = 15

    racer = WikiracerProblem()
    path = racer.wikiracer(source="/wiki/Computer_science", goal="/wiki/Richard_Soley")
    print(path)
    print(racer.internet.requests)
    print(len(racer.internet.requests))
    assert len(racer.internet.requests) <= limit
    
def test_wikiracer_2():
    limit = 80

    racer = WikiracerProblem()
    path = racer.wikiracer(source="/wiki/Waakirchen", goal="/wiki/A")
    print("\n", path)
    print(len(racer.internet.requests))
    assert len(racer.internet.requests) <= limit

def test_wikiracer_multiple():
    '''
    Tests wikiracer on multiple websites.
    Does a combination of all links and calls wikiracer to tests those combinations.
    This also tests that the path between pages is actually navigable
    Beware: This function test take long time to run. You can change to a smaller list to verify
    that your algorithm works.
    The default test takes around 5-10 minutes when the pages are not cached.
    '''
    pages = ['Jesus', 'Adolf_Hitler', 'Michael_Jordan', 'United_Nations', 'Kobe_Bryant', 'Brazil']

    allCombinations = list(combinations(pages, 2))
    paths = []
    with open("../results.txt", "w") as results:
        for combination in allCombinations:
            racer = WikiracerProblem()
            paths.append(racer.wikiracer(source=r'/wiki/' + combination[0], goal=r'/wiki/' + combination[1]))
            results.write(f"{combination} {len(racer.internet.requests)}\n")

        for i, path in enumerate(paths):
            results.write(f"\n####### {i} #######\n")
            source = path.pop(0)
            results.write(f"{source}\n")
            while len(path) > 0:
                destination = path.pop(0)
                results.write(f"{destination}\n")
                links = Parser.get_links_in_page(Internet().get_page(source))
                assert destination in links
                source = destination
            results.write(f"#################\n")

test_trivial()
test_wikiracer_1()
test_wikiracer_2()
# test_wikiracer_multiple()

In [47]:
import heapq

def dijkstra(graph, start, end):
    # Priority queue stores tuples of (distance, node)
    priority_queue = [(0, start)]
    
    # Dictionary to store the shortest distance found so far
    distances = {node: float('inf') for node in graph}
    distances[start] = 0
    
    # Dictionary to store the predecessor of each node
    predecessors = {node: None for node in graph}

    while priority_queue:
        current_distance, current_node = heapq.heappop(priority_queue)

        # If we reached the target, reconstruct and return the path
        if current_node == end:
            path = []
            node = end
            while node is not None:
                path.insert(0, node)
                node = predecessors[node]
            return distances[end], path

        # If the extracted distance is greater than the known shortest, skip
        if current_distance > distances[current_node]:
            continue

        for neighbor, weight in graph[current_node].items():
            distance = current_distance + weight
            
            # If a shorter path to the neighbor is found, update and push to queue
            if distance < distances[neighbor]:
                distances[neighbor] = distance
                predecessors[neighbor] = current_node
                heapq.heappush(priority_queue, (distance, neighbor))

    # If the loop finishes without finding the destination
    return float('inf'), None

graph = {
    'A': {'B': 1, 'C': 4},
    'B': {'A': 1, 'C': 2, 'D': 5},
    'C': {'A': 4, 'B': 2, 'D': 1},
    'D': {'B': 5, 'C': 1}
}

start_node = 'A'
end_node = 'D'

dijkstra(graph, start_node, end_node)
shortest_distance, shortest_path = dijkstra(graph, start_node, end_node)

if shortest_path:
    print(f"Shortest distance from {start_node} to {end_node}: {shortest_distance}")
    print(f"Shortest path: {' -> '.join(shortest_path)}")
else:
    print(f"No path found from {start_node} to {end_node}")


Shortest distance from A to D: 4
Shortest path: A -> B -> C -> D
